# Notebook to generate functions module

In [35]:
import inspect
import project

from mintalib import core


In [36]:
core.calc_eval

<cyfunction calc_eval at 0x10ee43c70>

In [37]:


def core_functions(first_param=('series', 'prices')):
    """ list of core functions """

    result = []
    
    for k, v in vars(core).items():
        if k.startswith("_") or not k.islower():
            continue

        if isinstance(v, type) or not callable(v):
            continue

        params = list(inspect.signature(v).parameters)

        if params[0] not in first_param:
            continue

        result.append(k)

    return result


print(core_functions())

['calc_avgprice', 'calc_typprice', 'calc_wclprice', 'calc_midprice', 'calc_price', 'crossover', 'crossunder', 'flag_above', 'flag_below', 'invert_flag', 'updown_flag', 'calc_log', 'calc_exp', 'calc_roc', 'calc_diff', 'calc_min', 'calc_max', 'calc_sum', 'calc_mad', 'calc_stdev', 'calc_sma', 'calc_ema', 'calc_rma', 'calc_wma', 'calc_dema', 'calc_tema', 'calc_ma', 'calc_rsi', 'calc_plusdi', 'calc_minusdi', 'calc_adx', 'calc_trange', 'calc_atr', 'calc_natr', 'calc_latr', 'calc_psar', 'calc_cci', 'calc_cmf', 'calc_mfi', 'calc_bop', 'calc_bbands', 'calc_keltner', 'efficiency_ratio', 'calc_kama', 'calc_macd', 'calc_ppo', 'calc_slope', 'calc_curve', 'calc_stoch', 'streak_up', 'streak_down', 'calc_eval']


In [38]:
def make_function(func, verbose=False):
    fname = func.__qualname__
    module = func.__module__.rpartition('.')[2]
    qname = f"{module}.{fname}"
    indent  = " " * 4

    signature = inspect.signature(func)
    parameters = list(signature.parameters.values())
    Parameter = inspect.Parameter

    if verbose:
        print("signature", signature)
        print("parameters", parameters)

    newparams = []
    extras = dict()
    ftype = parameters[0].name
    
    def annotate(p):
        if p.name in ['period']:
            return p.replace(annotation=int)

        if p.annotation == 'str':
            return p.replace(annotation=str)

        if type(p.default) in (int, float, bool):
            return p.replace(annotation=type(p.default))
        
        return p

    for i, p in enumerate(parameters):
        if p.name == 'wrap':
            # extras.update(wrap=True)
            continue

        p = annotate(p)
        newparams.append(p)

    if ftype == 'series':
        default = None
        p = Parameter('item', Parameter.KEYWORD_ONLY, default=default, annotation=str)
        newparams.append(p)

    if verbose:
        print("newparams", newparams)
        print("extras", extras)

    newsig = inspect.Signature(newparams)

    name = func.__name__.upper()
    if name.startswith("CALC_"):
        name = name.partition('_')[2]

    def argument(arg):
        return ftype if arg == ftype else f"{arg}={arg}"

    args = [argument(p.name) for p in newparams if p.name not in ['item']]
    args += [f"{k}={v!r}" for k, v in extras.items()]
    args = ", ".join(args)

    code = ""
    # code = "# noinspection PyPep8Naming\n"

    code += f"@wrap_function({qname})\n"
    code += f"def {name}" + str(newsig) + ":\n"

    if ftype == 'series':
        code += indent + "series = get_series(series, item=item)\n"

    code += indent + f"result = {qname}({args})\n"
    code += indent + f"return wrap_result(result, {ftype})\n"

    return code


print(make_function(core.calc_ema))
#print(make_function(core.calc_slope))
#print(make_function(core.calc_price))



@wrap_function(core.calc_ema)
def EMA(series, period: int, *, adjust: bool = False, item: str = None):
    series = get_series(series, item=item)
    result = core.calc_ema(series, period=period, adjust=adjust)
    return wrap_result(result, series)


In [39]:
def make_functions():
    buffer = "# Do not edit. File was generated by make-functions!\n\n"

    buffer += "from . import core\n"
    buffer += "from .utils import get_series, wrap_function, wrap_result\n\n"
    buffer += "nan: float = float('NAN')\n"
    buffer += "\n\n"

    names = core_functions()

    for name in names:
        func = getattr(core, name)
        text = make_function(func)
        buffer += text + "\n\n"

    buffer += "__all__ = [n for n in dir() if n.isupper()]\n"

    return buffer

code = make_functions()
print(code[:256], "...")





# Do not edit. File was generated by make-functions!

from . import core
from .utils import get_series, wrap_function, wrap_result

nan: float = float('NAN')


@wrap_function(core.calc_avgprice)
def AVGPRICE(prices):
    result = core.calc_avgprice(prices) ...


In [40]:
import importlib.util

def new_module(name):
    spec = importlib.util.spec_from_loader(name, None)
    return importlib.util.module_from_spec(spec)

functions = new_module("mintalib.indicators")

exec(code, functions.__dict__)

vars(functions)


{'__name__': 'mintalib.indicators',
 '__doc__': None,
 '__package__': 'mintalib',
 '__loader__': None,
 '__spec__': ModuleSpec(name='mintalib.indicators', loader=None),
 '__builtins__': {'__name__': 'builtins',
  '__doc__': "Built-in functions, exceptions, and other objects.\n\nNoteworthy: None is the `nil' object; Ellipsis represents `...' in slices.",
  '__package__': '',
  '__loader__': _frozen_importlib.BuiltinImporter,
  '__spec__': ModuleSpec(name='builtins', loader=<class '_frozen_importlib.BuiltinImporter'>),
  '__build_class__': <function __build_class__>,
  '__import__': <function __import__>,
  'abs': <function abs(x, /)>,
  'all': <function all(iterable, /)>,
  'any': <function any(iterable, /)>,
  'ascii': <function ascii(obj, /)>,
  'bin': <function bin(number, /)>,
  'breakpoint': <function breakpoint>,
  'callable': <function callable(obj, /)>,
  'chr': <function chr(i, /)>,
  'compile': <function compile(source, filename, mode, flags=0, dont_inherit=False, optimize=-1,

In [41]:
outfile = project.pkgdir / "functions.py"

print(f"Updating {outfile.name} ...")

outfile.write_text(code)


Updating functions.py ...


11066